<img src="https://www.colorado.edu/rc/sites/default/files/page/logo.png"
     alt="Logo for Research Computing @ University of Colorado Boulder"
     width="400" />
     
# Overview over `ipyparallel`

## Introduction

`IPython` supports many different styles of parallelism:
- Task parallelism
- Data parallel
- Single program, multiple data parallelism
- Multiple program, multiple data parallelism
- Combination of all approaches above


## Typical use cases

- Quickly parallelize algorithms that are embarrassingly parallel.
- Analyze and visualize large datasets interactively using matplotlib or other python libraries.
- Run a set of tasks on several nodes of a cluster using dynamic load balancing.
- Steer a MPI based simulation on a supercomputer from on IPython session on your laptop
- Develop, test, and debug parallel algorithms that use MPI interactively


## Architecture overview

When we start an `ipcluster` you generally start one controller process and several engines. The controller and engines do not necessarily run on the same node (or computer).

Most of the code will probably run on the controller, but you will run the compute intensive portions of your program on the engines.

The engines need to receive data and code from the controller:

<img src="./controller_send.png"
     alt="Controller with 2 engines sending data and code"
     width="300" />
    <img src="./controller_receive.png"
     alt="Controller with 2 engines receiving data"
     width="300" /> 

# Easy parallelism using `map`

## Starting an `ipcluster`

Using the cluster tab or

In [6]:
!ipcluster start -n 4 --daemonize

## Loading the right modules

In [2]:
import ipyparallel

## Create a client instance, used to connect the controller to the remote engines

In [3]:
rc = ipyparallel.Client(profile='example-shas')
nengines = len(rc)
nengines

14

In [4]:
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

## Create a `DirectView`

A `DirectView` is created by list access to a client instance:

In [5]:
dvall  = rc[:]
print(dvall)

<DirectView [0, 1, 2, 3,...]>


In [6]:
type(dvall)

ipyparallel.client.view.DirectView

## By default only the controller executes code

In [7]:
import socket
print('\n Running on',socket.gethostname())


 Running on shas0626


## Parallelizing the serial image example
Converting a list of images to grayscale

In [8]:
def convert2greyscale(path):
    from skimage.io import imread
    from skimage.color import rgb2gray
    img = imread(path)
    img_gray = rgb2gray(img)
    return img_gray

Note: importing the modules inside the functions avoids problems accessing modules on the engines. 
Other approaches are possible and will be discussed in later notebooks.

In [9]:
import os
pictures_dir = os.path.join('.', 'images', 'cornflower')
pictures = []
for directory, subdirs, files in os.walk(pictures_dir):
    for fname in files:
        if fname.lower().endswith(('.jpg', '.png')):
            pictures.append(os.path.join(directory, fname))

### The serial exampel for reference

In [13]:
%cd /home/thha0714/Projects/RMACC-2019-ipyparallel

/projects/thha0714/RMACC-2019-ipyparallel


In [10]:
%%time
sconverted = [*map(convert2greyscale, pictures[:32])]

CPU times: user 12.7 s, sys: 2.86 s, total: 15.6 s
Wall time: 16.2 s


### Parallelizing using the parallel map function 

In [11]:
%%time
pconverted = dvall.map_sync(convert2greyscale, pictures[:32])

CompositeError: one or more exceptions from call to method: convert2greyscale
[0:apply]: FileNotFoundError: [Errno 2] No such file or directory: './images/cornflower/Steuben-Parade-Cornflower-logo.jpg'
[1:apply]: FileNotFoundError: [Errno 2] No such file or directory: './images/cornflower/Cornflower.jpg'
[2:apply]: FileNotFoundError: [Errno 2] No such file or directory: './images/cornflower/Cornflower_Filagree_bead_MET_DT301244.jpg'
[3:apply]: FileNotFoundError: [Errno 2] No such file or directory: './images/cornflower/Harvestman%2C_aphids%2C_cornflower_%2820357954644%29.jpg'
.... 12 more exceptions ...

   ### Try again with a load-balanced view

In [17]:
dlb = rc.load_balanced_view()

In [19]:
%%time
lbconverted = dlb.map_sync(convert2greyscale, pictures[:32])

CPU times: user 148 ms, sys: 1.54 s, total: 1.68 s
Wall time: 8.66 s
